In [8]:
import pandas as pd
import json
import requests
from urllib import parse
# from elasticsearch import Elasticsearch
from os import name
import xml.etree.ElementTree as et
import bs4
from lxml import html
# from urllib.parse import urlencode, quote_plus, unquote
import xmltodict
import json


## api 설정 및 코드

In [2]:
api_name = ['getOcBysidoFireSmrzPcnd','getOcByfrstFireSmrzPcnd','getOcBysidoFpcnd','getOcBysidoFireOcrnPcnd','getOcFirePcnd',
            'getOcFirePrcnd','getOcArFireByplceFpcnd','getOcIgntnByfctrFpcnd','getOcIgntnBypntFpcnd','getOcIgntnByplceFpcnd',
            'getOcFrstBychlrdFpcnd','getOcStrcStrcstFpcnd','getOcRockByminfctyFpcnd','getOcVhclIgntnBypntFpcnd','getOcShipByarplFpcnd',
            'getOcBywdldFpcnd','getOcIgntnByahsFpcnd','getArFireByplceFpcnd','getOcRockMnfctyPcnd']
url_base = 'http://apis.data.go.kr/1661000/FireInformationService/'
key = 'WwrHmjfnt6hWvSuw%2FI%2FGOBndFtHW9C9MlIGXnN2O3LEhlG%2B2ZtVqjYXef9TiqpLftYFOzgag5A2NOAF1d4hx7A%3D%3D'

# 날짜는 2021-01-01 부터 ~ 2022-11-30

### 19개의 개별코드

In [237]:
# 1. 소방청 국가화재정보 시도단위화재발생현황 목록조회 (1day:3)sidoNm

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20220101'
})

url = url_base+api_name[0]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe[dataframe['sidoNm'].isin(['서울특별시','경기도','인천광역시'])].rename(
    columns={
        'sidoNm':'시도명', 
        'flsrpPrcsMnb':'오보처리건수', 
        'slfExtshMnb':'자체진화건수',
        'fireRcptMnb':'화재접수건수', 
        'stnEndMnb':'상황종료건수', 
        'ocrnYmd':'발생일자', 
        'falsDclrMnb':'허위신고건수'
        })

In [239]:
# 2. 소방청 국가화재정보 소방관서별화재발생현황 (1day:57)sidoHqFrstCetrNm

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20220101'
})

url = url_base+api_name[1]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe[dataframe['sidoHqFrstCetrNm'].isin(['서울소방재난본부','경기소방재난본부','인천소방안전본부'])].rename(
    columns={
        'sidoHqFrstCetrNm':'시도본부소방서센터명', 
        'fireProgMnb':'화재집행건수', 
        'flsrpPrcsMnb':'오보처리건수', 
        'slfExtshMnb':'자체진화건수',
        'fireRcptMnb':'화재접수건수', 
        'frstCetrNm':'소방서센터명', 
        'stnEndMnb':'상황종료건수', 
        'ocrnYmd':'발생일자', 
        'falsDclrMnb':'허위신고건수'
        })


In [233]:
# 3. 소방청 국가화재정보 시도별화재인명피해현황 목록조회 (1day:3)sidoNm

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20220101'
})

url = url_base+api_name[2]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe[dataframe['sidoNm'].isin(['서울특별시','경기도','인천광역시'])].rename(
    columns={
        'vctmPercnt':'사고자인원수', 
        'lifeDmgPercnt':'인명피해인원수', 
        'injrdprPercnt':'부상자인원수', 
        'sidoNm':'시도명',
        'ocrnMnb':'발생건수', 
        'ocrnYmd':'발생일자'
        })


In [231]:
# 4. 소방청 국가화재정보 시도별화재재산피해현황 목록조회 (1day:3)sidoNm

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20220101'
})

url = url_base+api_name[3]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe[dataframe['sidoNm'].isin(['서울특별시','경기도','인천광역시'])].rename(
    columns={
        'prptDmgSbttAmt':'재산피해소계금액', 
        'sidoNm':'시도명',
        'ocrnMnb':'발생건수', 
        'ocrnYmd':'발생일자'
        })


In [229]:
# 5. 소방청 국가화재정보 화재인명피해현황 목록조회(1day:1)

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20220101'
})

url = url_base+api_name[4]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe.rename(
    columns={
        'vctmPercnt':'사고자인원수', 
        'lifeDmgPercnt':'인명피해인원수',
        'injrdprPercnt':'부상자인원수',
        'ocrnMnb':'발생건수', 
        'ocrnYmd':'발생일자'
        })


In [228]:
# 6. 소방청 국가화재정보 화재재산피해현황 목록조회(1day:1)

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20220101'
})

url = url_base+api_name[5]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe.rename(
    columns={
        'prptDmgSbttAmt':'재산피해소계금액', 
        'restDmgAmt':'부동산피해금액',
        'ocrnMnb':'발생건수',
        'ocrnYmd':'발생일자', 
        'perpDmgAmt':'동산피해금액'
        })


In [224]:
# 7. 소방청 국가화재정보 화재장소별화재현황 목록조회(1day:?)

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20220101'
})

url = url_base+api_name[6]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe.rename(
    columns={
        'prptDmgSbttAmt':'재산피해소계금액', 
        'injrdprPercnt':'부상자인원수',
        'firePlceSctnNm':'화재장소구분명',
        'vctmPercnt':'사고자인원수',
        'lifeDmgPercnt':'인명피해인원수',
        'ocrnMnb':'발생건수',
        'ocrnYmd':'발생일자'
        })


In [247]:
# 8. 소방청 국가화재정보 발화요인별화재현황 목록조회(1day:?)

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20220101'
})

url = url_base+api_name[7]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe.rename(
    columns={
        'prptDmgSbttAmt':'재산피해소계금액', 
        'injrdprPercnt':'부상자인원수',
        'igntnFctrSclsfNm':'발화요인소분류명',
        'vctmPercnt':'사고자인원수',
        'ocrnMnb':'발생건수',
        'ocrnYmd':'발생일자',
        'igntnFctrLclsNm':'발화요인대분류명'
        })


In [219]:
# 9. 소방청 국가화재정보 발화지점별화재현황 목록조회(1day:?)

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20220101'
})

url = url_base+api_name[8]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe.rename(
    columns={
        'prptDmgSbttAmt':'재산피해소계금액', 
        'vctmPercnt':'사고자인원수',
        'lifeDmgPercnt':'인명피해인원수',
        'injrdprPercnt':'부상자인원수',
        'ocrnMnb':'발생건수',
        'ocrnYmd':'발생일자',
        'igntnBrofcLclsNm':'발화지점대분류명',
        'igntnBrofcSclsfNm':'발화지점소분류명'
        })


In [245]:
# 10. 소방청 국가화재정보 발화장소별화재현황 목록조회(1day:?)

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20220101'
})

url = url_base+api_name[9]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe.rename(
    columns={
        'prptDmgSbttAmt':'재산피해소계금액', 
        'injrdprPercnt':'부상자인원수',
        'igntnPlceMclsfNm':'발화장소중분류명',
        'igntnPlceLclsNm':'발화장소대분류명',
        'vctmPercnt':'사고자인원수',
        'lifeDmgPercnt':'인명피해인원수',
        'ocrnMnb':'발생건수',
        'ocrnYmd':'발생일자',
        })

In [241]:
# 11. 소방청 국가화재정보 최초착화물별화재현황 목록조회(1day:?)

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20220101'
})

url = url_base+api_name[10]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe.rename(
    columns={
        'prptDmgSbttAmt':'재산피해소계금액', 
        'frstChlrdSclsfNm':'최초착화물소분류명',
        'injrdprPercnt':'부상자인원수',
        'vctmPercnt':'사고자인원수',
        'lifeDmgPercnt':'인명피해인원수',
        'ocrnMnb':'발생건수',
        'ocrnYmd':'발생일자',
        'frstChlrdLclsNm':'최초착화물대분류명',
        })

In [213]:
# 12. 소방청 국가화재정보 건물구조별화재현황 목록조회(1day:?)

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20220101'
})

url = url_base+api_name[11]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe.rename(
    columns={
        'prptDmgSbttAmt':'재산피해소계금액', 
        'bldcnsNm':'건축구조명',
        'injrdprPercnt':'부상자인원수',
        'vctmPercnt':'사고자인원수',
        'ocrnMnb':'발생건수',
        'ocrnYmd':'발생일자',
        })

In [211]:
# 13. 소방청 국가화재정보 위험물제조소별화재현황 목록조회(1day:?)

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20070105'
})

url = url_base+api_name[12]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe.rename(
    columns={
        'prptDmgSbttAmt':'재산피해소계금액', 
        'injrdprPercnt':'부상자인원수',
        'rockMnfctySclsfNm':'위험물제조소소분류명',
        'vctmPercnt':'사고자인원수',
        'lifeDmgPercnt':'인명피해인원수',
        'rockMnfctyLclsNm':'위험물제조소대분류명',
        'ocrnMnb':'발생건수',
        'ocrnYmd':'발생일자',
        })

In [209]:
# 14. 소방청 국가화재정보 차량발화지점별화재현황 목록조회(1day:?)

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20070105'
})

url = url_base+api_name[13]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe.rename(
    columns={
        'prptDmgSbttAmt':'재산피해소계금액', 
        'vctmPercnt':'사고자인원수',
        'lifeDmgPercnt':'인명피해인원수',
        'injrdprPercnt':'부상자인원수',
        'ocrnMnb':'발생건수',
        'ocrnYmd':'발생일자',
        })

In [207]:
# 15. 소방청 국가화재정보 선박항공기별화재현황 목록조회(1day:?)

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20070118'
})

url = url_base+api_name[14]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe.rename(
    columns={
        'prptDmgSbttAmt':'재산피해소계금액', 
        'vctmPercnt':'사고자인원수',
        'lifeDmgPercnt':'인명피해인원수',
        'injrdprPercnt':'부상자인원수',
        'ocrnMnb':'발생건수',
        'ocrnYmd':'발생일자',
        'shipArplSclsfNm':'선박항공기소분류명',
        'shipArplLclsNm':'선박항공기대분류명',
        })

In [205]:
# 16. 소방청 국가화재정보 임야별화재현황 목록조회(1day:?)

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20070105'
})

url = url_base+api_name[15]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe.rename(
    columns={
        'prptDmgSbttAmt':'재산피해소계금액', 
        'vctmPercnt':'사고자인원수',
        'lifeDmgPercnt':'인명피해인원수',
        'injrdprPercnt':'부상자인원수',
        'ocrnMnb':'발생건수',
        'ocrnYmd':'발생일자',
        'wdldLclsNm':'임야대분류명',
        'wdldSclsfNm':'임야소분류명',
        })

In [203]:
# 17. 소방청 국가화재정보 발화열원별화재현황(1day:?)

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20070105'
})

url = url_base+api_name[16]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe.rename(
    columns={
        'prptDmgSbttAmt':'재산피해소계금액', 
        'injrdprPercnt':'부상자인원수', 
        'vctmPercnt':'사고자인원수', 
        'lifeDmgPercnt':'인명피해인원수', 
        'ocrnMnb':'발생건수',
        'ocrnYmd':'발생일자',
        'igntnAhsSclsfNm':'발화열원소분류명',
        'igntnAhsLclsNm':'발화열원대분류명',
        })

In [201]:
# 18. 소방청 국가화재정보 지역별화재장소별화재발생현황(1day:?)sidoNm

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("ocrn_ymd"): '20070105'
})

url = url_base+api_name[17]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe[dataframe.sidoNm.isin(['서울특별시','경기도','인천광역시'])].rename(
    columns={
        'prptDmgSbttAmt':'재산피해소계금액', 
        'injrdprPercnt':'부상자인원수',
        'firePlceSctnNm':'화재장소구분명',
        'sidoNm':'시도명',
        'vctmPercnt':'사고자인원수',
        'lifeDmgPercnt':'인명피해인원수',
        'ocrnMnb':'발생건수',
        'ocrnYmd':'발생일자',
        })

In [199]:
# 19. 소방청 국가화재정보 위험물제조소등현황(1day:?)sidoNm

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("yr"): '2009',
})

url = url_base+api_name[18]
response = requests.get(url +  queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)    

df = dataframe[dataframe.sidoNm.isin(['서울특별시','경기도','인천광역시'])].rename(
    columns={
        'yr':'연도', 
        'rockMnfctyLclsNm':'위험물제조소대분류코드',
        'rockMnfctySclsfNm':'위험물제조소소분류코드',
        'sidoNm':'시도명',
        'rockMnfctyCnt':'위험물제조소수',
        })

In [248]:
df.columns

Index(['재산피해소계금액', '부상자인원수', '발화요인소분류명', '사고자인원수', '발생건수', '발생일자', '발화요인대분류명'], dtype='object')

In [240]:
df

,시도본부소방서센터명,화재집행건수,오보처리건수,자체진화건수,화재접수건수,소방서센터명,상황종료건수,발생일자,허위신고건수
0,서울소방재난본부,0,2,0,2,동작소방서,0,20220101,0
1,서울소방재난본부,0,1,0,2,종로소방서,1,20220101,0
2,서울소방재난본부,0,2,0,2,중부소방서,0,20220101,0
3,서울소방재난본부,0,2,0,3,광진소방서,1,20220101,0
4,서울소방재난본부,0,3,0,4,용산소방서,1,20220101,0
5,서울소방재난본부,0,3,1,4,동대문소방서,0,20220101,0
6,서울소방재난본부,0,1,0,1,영등포소방서,0,20220101,0
7,서울소방재난본부,0,1,1,2,성북소방서,0,20220101,0
8,서울소방재난본부,1,0,0,2,도봉소방서,1,20220101,0
9,서울소방재난본부,0,0,1,2,노원소방서,1,20220101,0


### 함수화

In [39]:
# 1~18
def api_func_1(num, date):

    queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
    parse.quote_plus("pageNo"): '1',
    parse.quote_plus("numOfRows"):'1000',
    parse.quote_plus("resultType"): 'json',
    parse.quote_plus("ocrn_ymd"): date
    })

    url = url_base+api_name[num-1]
    response = requests.get(url +  queryParams).text
    json_ob = json.loads(response)
    body = json_ob['response']['body']['items']['item']
    dataframe = pd.json_normalize(body)    
    
    if num == 1: 
        df = dataframe[dataframe['sidoNm'].isin(['서울특별시','경기도','인천광역시'])].rename(
            columns={
                'sidoNm':'시도명', 
                'flsrpPrcsMnb':'오보처리건수', 
                'slfExtshMnb':'자체진화건수',
                'fireRcptMnb':'화재접수건수', 
                'stnEndMnb':'상황종료건수', 
                'ocrnYmd':'발생일자', 
                'falsDclrMnb':'허위신고건수'
                })
    elif num == 2:
        df = dataframe[dataframe['sidoHqFrstCetrNm'].isin(['서울소방재난본부','경기소방재난본부','인천소방안전본부'])].rename(
            columns={
                'sidoHqFrstCetrNm':'시도본부소방서센터명', 
                'fireProgMnb':'화재집행건수', 
                'flsrpPrcsMnb':'오보처리건수', 
                'slfExtshMnb':'자체진화건수',
                'fireRcptMnb':'화재접수건수', 
                'frstCetrNm':'소방서센터명', 
                'stnEndMnb':'상황종료건수', 
                'ocrnYmd':'발생일자', 
                'falsDclrMnb':'허위신고건수'
                })
    elif num == 3:
        df = dataframe[dataframe['sidoNm'].isin(['서울특별시','경기도','인천광역시'])].rename(
            columns={
                'vctmPercnt':'사고자인원수', 
                'lifeDmgPercnt':'인명피해인원수', 
                'injrdprPercnt':'부상자인원수', 
                'sidoNm':'시도명',
                'ocrnMnb':'발생건수', 
                'ocrnYmd':'발생일자'
                })  
    elif num == 4:
        df = dataframe[dataframe['sidoNm'].isin(['서울특별시','경기도','인천광역시'])].rename(
            columns={
                'prptDmgSbttAmt':'재산피해소계금액', 
                'sidoNm':'시도명',
                'ocrnMnb':'발생건수', 
                'ocrnYmd':'발생일자'
                })
    elif num == 5:
        df = dataframe.rename(
            columns={
                'vctmPercnt':'사고자인원수', 
                'lifeDmgPercnt':'인명피해인원수',
                'injrdprPercnt':'부상자인원수',
                'ocrnMnb':'발생건수', 
                'ocrnYmd':'발생일자'
                })
    elif num == 6:
        df = dataframe.rename(
            columns={
                'prptDmgSbttAmt':'재산피해소계금액', 
                'restDmgAmt':'부동산피해금액',
                'ocrnMnb':'발생건수',
                'ocrnYmd':'발생일자', 
                'perpDmgAmt':'동산피해금액'
                })
    elif num == 7:
        df = dataframe.rename(
            columns={
                'prptDmgSbttAmt':'재산피해소계금액', 
                'injrdprPercnt':'부상자인원수',
                'firePlceSctnNm':'화재장소구분명',
                'vctmPercnt':'사고자인원수',
                'lifeDmgPercnt':'인명피해인원수',
                'ocrnMnb':'발생건수',
                'ocrnYmd':'발생일자'
                })
    elif num == 8:
        df = dataframe.rename(
            columns={
                'prptDmgSbttAmt':'재산피해소계금액', 
                'injrdprPercnt':'부상자인원수',
                'igntnFctrSclsfNm':'발화요인소분류명',
                'vctmPercnt':'사고자인원수',
                'ocrnMnb':'발생건수',
                'ocrnYmd':'발생일자',
                'igntnFctrLclsNm':'발화요인대분류명'
                })
    elif num == 9:
        try:
            df = dataframe.rename(
                columns={
                    'prptDmgSbttAmt':'재산피해소계금액', 
                    'vctmPercnt':'사고자인원수',
                    'lifeDmgPercnt':'인명피해인원수',
                    'injrdprPercnt':'부상자인원수',
                    'ocrnMnb':'발생건수',
                    'ocrnYmd':'발생일자',
                    'igntnBrofcLclsNm':'발화지점대분류명',
                    'igntnBrofcSclsfNm':'발화지점소분류명'
                    })
        except:
            pass
    elif num == 10:
        try:
            df = dataframe.rename(
                columns={
                    'prptDmgSbttAmt':'재산피해소계금액', 
                    'injrdprPercnt':'부상자인원수',
                    'igntnPlceMclsfNm':'발화장소중분류명',
                    'igntnPlceLclsNm':'발화장소대분류명',
                    'vctmPercnt':'사고자인원수',
                    'lifeDmgPercnt':'인명피해인원수',
                    'ocrnMnb':'발생건수',
                    'ocrnYmd':'발생일자',
                    })
        except:
            pass
    elif num == 11:
        try: 
            df = dataframe.rename(
                columns={
                    'prptDmgSbttAmt':'재산피해소계금액', 
                    'frstChlrdSclsfNm':'최초착화물소분류명',
                    'injrdprPercnt':'부상자인원수',
                    'vctmPercnt':'사고자인원수',
                    'lifeDmgPercnt':'인명피해인원수',
                    'ocrnMnb':'발생건수',
                    'ocrnYmd':'발생일자',
                    'frstChlrdLclsNm':'최초착화물대분류명',
                    })
        except:
            pass
    elif num == 12:
        try:
            df = dataframe.rename(
                columns={
                    'prptDmgSbttAmt':'재산피해소계금액', 
                    'bldcnsNm':'건축구조명',
                    'injrdprPercnt':'부상자인원수',
                    'vctmPercnt':'사고자인원수',
                    'ocrnMnb':'발생건수',
                    'ocrnYmd':'발생일자',
                    })
        except:
            pass
    elif num == 13:
        try:
            df = dataframe.rename(
                columns={
                    'prptDmgSbttAmt':'재산피해소계금액', 
                    'injrdprPercnt':'부상자인원수',
                    'rockMnfctySclsfNm':'위험물제조소소분류명',
                    'vctmPercnt':'사고자인원수',
                    'lifeDmgPercnt':'인명피해인원수',
                    'rockMnfctyLclsNm':'위험물제조소대분류명',
                    'ocrnMnb':'발생건수',
                    'ocrnYmd':'발생일자',
                    })
        except:
            pass
    elif num == 14:
        try:
            df = dataframe.rename(
                columns={
                    'prptDmgSbttAmt':'재산피해소계금액', 
                    'vctmPercnt':'사고자인원수',
                    'lifeDmgPercnt':'인명피해인원수',
                    'injrdprPercnt':'부상자인원수',
                    'ocrnMnb':'발생건수',
                    'ocrnYmd':'발생일자',
                    })
        except:
            pass
    elif num == 15:
        try:
            df = dataframe.rename(
                columns={
                    'prptDmgSbttAmt':'재산피해소계금액', 
                    'vctmPercnt':'사고자인원수',
                    'lifeDmgPercnt':'인명피해인원수',
                    'injrdprPercnt':'부상자인원수',
                    'ocrnMnb':'발생건수',
                    'ocrnYmd':'발생일자',
                    'shipArplSclsfNm':'선박항공기소분류명',
                    'shipArplLclsNm':'선박항공기대분류명',
                    })
        except:
            pass
    elif num == 16:
        try:
            df = dataframe.rename(
                columns={
                    'prptDmgSbttAmt':'재산피해소계금액', 
                    'vctmPercnt':'사고자인원수',
                    'lifeDmgPercnt':'인명피해인원수',
                    'injrdprPercnt':'부상자인원수',
                    'ocrnMnb':'발생건수',
                    'ocrnYmd':'발생일자',
                    'wdldLclsNm':'임야대분류명',
                    'wdldSclsfNm':'임야소분류명',
                    })
        except:
            pass
    elif num == 17:
        try:
            df = dataframe.rename(
                columns={
                    'prptDmgSbttAmt':'재산피해소계금액', 
                    'injrdprPercnt':'부상자인원수', 
                    'vctmPercnt':'사고자인원수', 
                    'lifeDmgPercnt':'인명피해인원수', 
                    'ocrnMnb':'발생건수',
                    'ocrnYmd':'발생일자',
                    'igntnAhsSclsfNm':'발화열원소분류명',
                    'igntnAhsLclsNm':'발화열원대분류명',
                    })
        except:
            pass
    elif num == 18:
        try:
            df = dataframe.rename(
                columns={
                    'prptDmgSbttAmt':'재산피해소계금액', 
                    'injrdprPercnt':'부상자인원수',
                    'firePlceSctnNm':'화재장소구분명',
                    'sidoNm':'시도명',
                    'vctmPercnt':'사고자인원수',
                    'lifeDmgPercnt':'인명피해인원수',
                    'ocrnMnb':'발생건수',
                    'ocrnYmd':'발생일자',
                    })
        except:
            pass
    return df

In [65]:
# 19
def api_func_2(date):

    queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
    parse.quote_plus("pageNo"): '1',
    parse.quote_plus("numOfRows"):'1000',
    parse.quote_plus("resultType"): 'json',
    parse.quote_plus("yr"): date,
    })

    url = url_base+api_name[18]
    response = requests.get(url +  queryParams).text
    json_ob = json.loads(response)
    body = json_ob['response']['body']['items']['item']
    dataframe = pd.json_normalize(body)    

    df = dataframe[dataframe.sidoNm.isin(['서울특별시','경기도','인천광역시'])].rename(
        columns={
            'yr':'연도', 
            'rockMnfctyLclsNm':'위험물제조소대분류코드',
            'rockMnfctySclsfNm':'위험물제조소소분류코드',
            'sidoNm':'시도명',
            'rockMnfctyCnt':'위험물제조소수',
            })
    return df

date_list2 = ['2021','2022']
box = pd.concat([api_func_2(2021),api_func_2(2022)])
box

In [4]:
start_date='20210101'
end_date='20221130'
date_list=pd.date_range(start=start_date, end=end_date, freq='D')
date_list = date_list.astype('str')
date_list=date_list.str.replace(pat=r'[^\w]',repl=r'',regex=True)
date_list=date_list.tolist()

In [62]:
n = 17

for i in date_list2:
    if i == date_list2[0]:
        box = api_func_2(n,i)
    else:
        box = pd.concat([box,api_func_2(n,i)])

box.info()
# box.to_csv('./box'+n+'.csv',mode = 'w', index=False)

ValueError: All objects passed were None

In [68]:
box.to_csv('./box19.csv',mode = 'w', index=False)

# 소방청 구급정보

In [ ]:
url = 'http://apis.data.go.kr/1661000/EmergencyInformationService/getEmgPatientTransferInfo'
key = 'WwrHmjfnt6hWvSuw%2FI%2FGOBndFtHW9C9MlIGXnN2O3LEhlG%2B2ZtVqjYXef9TiqpLftYFOzgag5A2NOAF1d4hx7A%3D%3D'


queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("sidoHqOgidNm"): '서울소방재난본부' ,
parse.quote_plus("stmtYm"): '202101'
})

response = requests.get(url + queryParams).text
json_ob = json.loads(response)

In [ ]:
body = json_ob['body']['items']
dataframe = pd.json_normalize(body)    

In [ ]:
dataframe

,sidoHqOgidNm,frnrAt,ptntSdtSeCdNm,anmlInctCdNm,stmtYm,ruptOccrPlcCdNm,rsacGutFsttOgidNm,ptntTyCdNm,ptntAge,rlifAcdAsmCdNm,stmtHh,rlifOccrTyCdNm,wmhtDamgCdNm
0,서울소방재난본부,N,남,None,202101,집,구로소방서,무직,79,None,20,질병,None
1,서울소방재난본부,N,여,None,202101,오락/문화시설,송파소방서,학생,9,None,20,질병,None
2,서울소방재난본부,N,여,None,202101,None,송파소방서,None,59,None,14,질병,None
3,서울소방재난본부,N,남,None,202101,집,송파소방서,무직,89,None,05,질병,None
4,서울소방재난본부,N,여,None,202101,상업시설,송파소방서,직장인,39,None,15,질병,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,서울소방재난본부,N,여,None,202101,기타,구로소방서,무직,49,낙상,17,기타,None
996,서울소방재난본부,N,여,None,202101,기타,구로소방서,무직,49,낙상,17,기타,None
997,서울소방재난본부,N,여,None,202101,상업시설,구로소방서,자영업,29,None,21,질병,None
998,서울소방재난본부,N,남,None,202101,집,구로소방서,무직,49,None,03,질병,None


# 소방청 구급통계서비스

In [ ]:
url = 'http://apis.data.go.kr/1661000/EmergencyStatisticsService/getTrafficAccidentEmgActStats'
key = 'WwrHmjfnt6hWvSuw%2FI%2FGOBndFtHW9C9MlIGXnN2O3LEhlG%2B2ZtVqjYXef9TiqpLftYFOzgag5A2NOAF1d4hx7A%3D%3D'


queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'1000',
parse.quote_plus("resultType"): 'json',
parse.quote_plus("sidoHqOgidNm"): '서울소방재난본부' ,
parse.quote_plus("stmtYm"): '202101'
})

response = requests.get(url + queryParams).text
json_ob = json.loads(response)

In [ ]:
body = json_ob['body']['items']
dataframe = pd.json_normalize(body)    

In [ ]:
dataframe

,gutCo,rsacGutFsttOgidNm,trnfPcnt,rcptYm,sidoHqOgidNm,rlifAcdAsmCdNm,rlifOccrTyCdNm,trnfCo
0,5,관악소방서,6,201908,서울소방재난본부,기타탈것,기타,5
1,2,성동소방서,2,202002,서울소방재난본부,기타탈것,기타,2
2,7,성동소방서,7,202002,서울소방재난본부,동승자,기타,7
3,19,성동소방서,18,202002,서울소방재난본부,보행자,기타,18
4,17,성동소방서,15,202002,서울소방재난본부,운전자,기타,15
...,...,...,...,...,...,...,...,...
995,4,동대문소방서,5,201907,서울소방재난본부,동승자,기타,4
996,25,동대문소방서,24,201907,서울소방재난본부,보행자,기타,24
997,16,용산소방서,16,201907,서울소방재난본부,보행자,기타,16
998,14,노원소방서,8,201907,서울소방재난본부,운전자,기타,8


# 소방청 소방항공기 보유현황

In [ ]:
url = 'http://apis.data.go.kr/1750000/fireAirService/getFireAirList'
key = 'WwrHmjfnt6hWvSuw%2FI%2FGOBndFtHW9C9MlIGXnN2O3LEhlG%2B2ZtVqjYXef9TiqpLftYFOzgag5A2NOAF1d4hx7A%3D%3D'


queryParams = '?' + parse.urlencode({
parse.quote_plus("pageNo"): '1',
parse.quote_plus("numOfRows"):'10'
}) + f'&{parse.quote_plus("serviceKey")}={key}'

response = requests.get(url + queryParams).text

In [ ]:
print(response)

<OpenAPI_ServiceResponse>
	<cmmMsgHeader>
		<errMsg>SERVICE ERROR</errMsg>
		<returnAuthMsg>HTTP ROUTING ERROR</returnAuthMsg>
		<returnReasonCode>04</returnReasonCode>
	</cmmMsgHeader>
</OpenAPI_ServiceResponse>


# 기상청 api
## 일자료

In [30]:
url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
key = 'qIlLlTZnU9ipgBO8frY4eIlcT5n%2BeoMENWFJFtdpD0oWo7kG4nEtn5eyxLs50fBoULGJMSW2oDPXQxgyyMurtQ%3D%3D'


queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("numOfRows"): 800,
parse.quote_plus("pageNo"): 1,
parse.quote_plus("dataType"): 'JSON',
parse.quote_plus("dataCd"): 'ASOS',
parse.quote_plus("dateCd"): 'DAY',
parse.quote_plus("startDt"): '20210101' ,
parse.quote_plus("endDt"): '20221130',
parse.quote_plus("stnIds"): 201
# 관측기관 지점코드 
# 수도권 : 98,99,102,108,112,119,202,203,201 
})

response = requests.get(url + queryParams).text
json_ob = json.loads(response)
body = json_ob['response']['body']['items']['item']
dataframe = pd.json_normalize(body)
dataframe

,stnId,stnNm,tm,avgTa,minTa,minTaHrmt,maxTa,maxTaHrmt,mi10MaxRn,mi10MaxRnHrmt,...,avgM05Te,avgM10Te,avgM15Te,avgM30Te,avgM50Te,sumLrgEv,sumSmlEv,n99Rn,iscs,sumFogDur
0,201,강화,2021-01-01,-4.9,-11.8,0331,0.8,1538,,,...,,,,,,,,,,
1,201,강화,2021-01-02,-5.3,-10.8,0738,-1.7,1510,,,...,,,,,,,,,,
2,201,강화,2021-01-03,-6.8,-11.2,0746,-2.2,1140,,,...,,,,,,,,,,
3,201,강화,2021-01-04,-4.6,-11.1,0656,-0.7,1543,,,...,,,,,,,,,,
4,201,강화,2021-01-05,-6.4,-12.0,2354,-2.8,0001,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,201,강화,2022-11-26,4.9,-2.5,2312,11.7,0001,,,...,,,,,,,,,,
695,201,강화,2022-11-27,3.5,-3.8,0448,10.0,1504,,,...,,,,,,,,,,
696,201,강화,2022-11-28,12.3,5.7,0001,17.2,1330,,,...,,,,,,,,,,
697,201,강화,2022-11-29,5.0,-2.8,2359,14.8,0033,,,...,,,,,,,,,,


In [31]:
# list_ID = [98,99,102,108,112,119,202,203,201]
# df = pd.DataFrame()

df = pd.concat([df,dataframe])
df.info()           


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6291 entries, 0 to 698
Data columns (total 62 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   stnId           6291 non-null   object
 1   stnNm           6291 non-null   object
 2   tm              6291 non-null   object
 3   avgTa           6291 non-null   object
 4   minTa           6291 non-null   object
 5   minTaHrmt       6291 non-null   object
 6   maxTa           6291 non-null   object
 7   maxTaHrmt       6291 non-null   object
 8   mi10MaxRn       6291 non-null   object
 9   mi10MaxRnHrmt   6291 non-null   object
 10  hr1MaxRn        6291 non-null   object
 11  hr1MaxRnHrmt    6291 non-null   object
 12  sumRnDur        6291 non-null   object
 13  sumRn           6291 non-null   object
 14  maxInsWs        6291 non-null   object
 15  maxInsWsWd      6291 non-null   object
 16  maxInsWsHrmt    6291 non-null   object
 17  maxWs           6291 non-null   object
 18  maxWsWd  

In [32]:
df.to_csv('./weather_api.csv',mode = 'w', index=False)

## 시간자료
안됨이거 오류남

In [140]:
url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
key = 'qIlLlTZnU9ipgBO8frY4eIlcT5n%2BeoMENWFJFtdpD0oWo7kG4nEtn5eyxLs50fBoULGJMSW2oDPXQxgyyMurtQ%3D%3D'


queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("numOfRows"): 100,
parse.quote_plus("pageNo"): 1,
parse.quote_plus("dataType"): 'JSON',
parse.quote_plus("dataCd"): 'ASOS',
parse.quote_plus("dateCd"): 'HR',
parse.quote_plus("startDt"): '20220101' ,
parse.quote_plus("startHh"): '01',
parse.quote_plus("endDt"): '20220601',
parse.quote_plus("endHh"): '01',
parse.quote_plus("stnIds"): 108 
# 관측기관 지점코드 
# 수도권 : 98,99,102,108,112,119,202,203,201 
})

response = requests.get(url + queryParams).text
json_ob = json.loads(response)
# body = json_ob['response']['body']['items']['item']
# dataframe = pd.json_normalize(body)

# 의료데이터

## 전국 응급의료기관 조회 서비스
### 응급실 실시간 가용병상조회
2개씩밖에안나옴 버리자 <br>
https://www.data.go.kr/iim/api/selectAPIAcountView.do

In [22]:
url = 'http://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire'
key = 'qIlLlTZnU9ipgBO8frY4eIlcT5n%2BeoMENWFJFtdpD0oWo7kG4nEtn5eyxLs50fBoULGJMSW2oDPXQxgyyMurtQ%3D%3D'

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
parse.quote_plus("STAGE1"): '서울특별시',
parse.quote_plus("STAGE2"): '서대문구',
parse.quote_plus("numOfRows"): 100,
parse.quote_plus("pageNo"): 1,
})

# xml로 받아온 response
response = requests.get(url + queryParams).text
# xmltodict로 dictionary형태로 바꿔줌
jsonString = json.dumps(xmltodict.parse(response), indent=4)
# dictiondary to json
dic = json.loads(jsonString)

body = dic['response']['body']['items']['item']
dataframe = pd.json_normalize(body)

### 응급의료기관 목록정보 조회

In [163]:
url = 'http://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytListInfoInqire'
key = 'qIlLlTZnU9ipgBO8frY4eIlcT5n%2BeoMENWFJFtdpD0oWo7kG4nEtn5eyxLs50fBoULGJMSW2oDPXQxgyyMurtQ%3D%3D'

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
#parse.quote_plus("STAGE1"): '서울특별시',
#parse.quote_plus("STAGE2"): '마포구',
parse.quote_plus("numOfRows"): 500,
parse.quote_plus("pageNo"): 1,
})

# xml로 받아온 response
response = requests.get(url + queryParams).text
# xmltodict로 dictionary형태로 바꿔줌
jsonString = json.dumps(xmltodict.parse(response), indent=4)
# dictiondary to json
dic = json.loads(jsonString)

body = dic['response']['body']['items']['item']
dataframe = pd.json_normalize(body)

In [164]:
dataframe

,dutyAddr,dutyEmcls,dutyEmclsName,dutyName,dutyTel1,dutyTel3,hpid,phpid,rnum,wgs84Lat,wgs84Lon
0,부산광역시 기장군 기장읍 대청로72번길 6,G007,지역응급의료기관,(의)서일의료재단기장병원,051-723-0171,051-723-2119,A1200028,A1200028,1,35.23602946449906,129.21649161387128
1,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",G007,지역응급의료기관,(의)성세의료재단 뉴성민병원,032-726-1000,032-726-1190,A1400008,A1400008,2,37.5089936801167,126.669478649026
2,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",G006,지역응급의료센터,(의)영문의료재단다보스병원,031-8021-2000,031-8021-2130,A2100019,A2100019,3,37.234641294534285,127.21049868474802
3,경기도 용인시 처인구 고림로 81 (고림동),G007,지역응급의료기관,(의)효심의료재단용인서울병원,031-337-0114,031-336-0119,A2100122,A2100122,4,37.240316373,127.2144914405
4,"대구광역시 수성구 달구벌대로 3190 (신매동, 천주성삼병원)",G007,지역응급의료기관,(재)미리내천주성삼성직수도회천주성삼병원,053-790-1000,053-790-0555,A1300081,A1300081,5,35.84022544517349,128.70573339664242
...,...,...,...,...,...,...,...,...,...,...,...
404,강원도 화천군 화천읍 강변로 111 (화천군보건의료원),G007,지역응급의료기관,화천군보건의료원,033-442-7115,033-440-2880,E2200025,E2200025,405,38.1073824571151,127.708883438378
405,강원도 횡성군 횡성읍 횡성로 275,G007,지역응급의료기관,횡성대성병원,033-343-0901,033-345-1191,A2200016,A2200016,406,37.47845901170136,127.97994775890871
406,"경기도 안양시 만안구 삼덕로 9 (안양동, 안양샘병원)",G006,지역응급의료센터,효산의료재단안양샘병원,031-467-9717,031-467-9119,A2100015,A2100015,407,37.39340413136221,126.92447734066778
407,"경기도 군포시 군포로 591 (당동, (G샘병원)군포샘병원)",G006,지역응급의료센터,효산의료재단지샘병원,031-389-3000,031-389-3119,A2100054,A2100054,408,37.35864464913468,126.94735972779895


## 응급의료기관 기본정보 조회

In [113]:
url = 'http://apis.data.go.kr/B552657/ErmctInfoInqireService/getEgytBassInfoInqire'
key = 'qIlLlTZnU9ipgBO8frY4eIlcT5n%2BeoMENWFJFtdpD0oWo7kG4nEtn5eyxLs50fBoULGJMSW2oDPXQxgyyMurtQ%3D%3D'

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
# parse.quote_plus("HPID"): 'A0000028', # 기관ID
parse.quote_plus("pageNo"): 1,
parse.quote_plus("numOfRows"): 1000,
})

# xml로 받아온 response
response = requests.get(url + queryParams).text
# xmltodict로 dictionary형태로 바꿔줌
jsonString = json.dumps(xmltodict.parse(response), indent=4)
# dictiondary to json
dic = json.loads(jsonString)

body = dic['response']['body']['items']['item']
dataframe = pd.json_normalize(body)

## 외상센터 목록정보 조회

In [4]:
url = 'http://apis.data.go.kr/B552657/ErmctInfoInqireService/getStrmListInfoInqire'
key = 'qIlLlTZnU9ipgBO8frY4eIlcT5n%2BeoMENWFJFtdpD0oWo7kG4nEtn5eyxLs50fBoULGJMSW2oDPXQxgyyMurtQ%3D%3D'

queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
#parse.quote_plus("STAGE1"): '서울특별시',
#parse.quote_plus("STAGE2"): '마포구',
parse.quote_plus("numOfRows"): 100,
# parse.quote_plus("pageNo"): 1,
})

# xml로 받아온 response
response = requests.get(url + queryParams).text
# xmltodict로 dictionary형태로 바꿔줌
jsonString = json.dumps(xmltodict.parse(response), indent=4)
# dictiondary to json
dic = json.loads(jsonString)

body = dic['response']['body']['items']['item']
dataframe = pd.json_normalize(body)

In [6]:
dataframe

,dutyAddr,dutyEmcls,dutyEmclsName,dutyName,dutyTel1,dutyTel3,hpid,phpid,rnum,wgs84Lat,wgs84Lon
0,"서울특별시 성북구 고려대로 73, 고려대병원 (안암동5가)",G001,권역응급의료센터,(학)고려대학교의과대학부속병원(안암병원),1577-0083,02-920-5374,A1100008,A1100008,1,37.58715608002366,127.02647086385966
1,"경기도 의정부시 천보로 271, 의정부성모병원 (금오동)",G001,권역응급의료센터,가톨릭대학교의정부성모병원,1661-7500,031-820-5200,A2100040,A2100040,2,37.75865547828227,127.07767290194454
2,"대구광역시 중구 동덕로 130 (삼덕동2가, 경북대학교병원)",G001,권역응급의료센터,경북대학교병원,053-200-5114,053-200-5100,A1300002,A1300002,3,35.866235497221617,128.60431466100362
3,경상남도 진주시 강남로 79 (칠암동),G001,권역응급의료센터,경상국립대학교병원,055-750-8000,055-750-8282,A2800001,A2800001,4,35.176340165507916,128.09564813723503
4,"서울특별시 구로구 구로동로 148, 고려대부속구로병원 (구로동)",G001,권역응급의료센터,고려대학교의과대학부속구로병원,02-2626-1114,02-2626-1550,A1100014,A1100014,5,37.49211114525054,126.8847449363546
5,"충청남도 천안시 동남구 망향로 201 (안서동, 단국대학교의과대학부속병원)",G001,권역응급의료센터,단국대학교의과대학부속병원,041-550-7114,041-550-6840,A2400002,A2400002,6,36.84295215169462,127.17327537305154
6,전라남도 목포시 영산로 483 (상동),G001,권역응급의료센터,목포한국병원,061-270-5500,061-270-5666,A2600011,A2600011,7,34.8092145829474,126.4163391650001
7,부산광역시 서구 구덕로 179 (아미동1가),G006,지역응급의료센터,부산대학교병원,051-240-7000,051-240-7776,A1200002,A1200002,8,35.10105418267188,129.01922207984205
8,서울특별시 종로구 대학로 101 (연건동),G001,권역응급의료센터,서울대학교병원,1588-5700,02-2072-2475,A1100017,A1100017,9,37.57966608924356,126.99896308412191
9,경기도 수원시 영통구 월드컵로 164 (원천동),G001,권역응급의료센터,아주대학교병원,031-219-5114,031-219-7700,A2100002,A2100002,10,37.2794477573662,127.04752107106702
